# E01 가위바위보 분류기

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import glob
from PIL import Image

In [2]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

In [3]:
def load_data(img_path, number_of_data=6777):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

In [11]:
# Modeling
n_channel_1=128
n_channel_2=256
n_dense=32
n_train_epoch=8

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 128)       3584      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 256)       295168    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                204832    
_________________________________________________________________
dense_3 (Dense)              (None, 3)                

In [15]:
#이미지 리사이즈
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
resize_images(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
resize_images(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
resize_images(image_dir_path)
print("이미지 resize 완료!")


1210  images to be resized.
1210  images resized.
1228  images to be resized.
1228  images resized.
1311  images to be resized.
1311  images resized.
이미지 resize 완료!


In [16]:
#이미지 데이터 정규화
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 3749 입니다.
x_train shape: (6777, 28, 28, 3)
y_train shape: (6777,)


In [17]:
model.fit(x_train_norm, y_train, epochs=n_train_epoch)

Epoch 1/8
212/212 [==============================] - 1s 4ms/step - loss: 0.0992 - accuracy: 0.9739
Epoch 2/8
212/212 [==============================] - 1s 4ms/step - loss: 0.0625 - accuracy: 0.9898
Epoch 3/8
212/212 [==============================] - 1s 4ms/step - loss: 0.0587 - accuracy: 0.9904
Epoch 4/8
212/212 [==============================] - 1s 4ms/step - loss: 0.0635 - accuracy: 0.9872
Epoch 5/8
212/212 [==============================] - 1s 4ms/step - loss: 0.0563 - accuracy: 0.9892
Epoch 6/8
212/212 [==============================] - 1s 4ms/step - loss: 0.0435 - accuracy: 0.9937
Epoch 7/8
212/212 [==============================] - 1s 4ms/step - loss: 0.0256 - accuracy: 0.9991
Epoch 8/8
212/212 [==============================] - 1s 4ms/step - loss: 0.0212 - accuracy: 0.9994


In [8]:
#테스트 데이터 만들기
#테스트 이미지 리사이즈
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/paper"
resize_images(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/scissor"
resize_images(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/rock"
resize_images(image_dir_path)
print("이미지 resize 완료!")


108  images to be resized.
108  images resized.
100  images to be resized.
100  images resized.
104  images to be resized.
104  images resized.
이미지 resize 완료!


In [9]:
#테스트 이미지 데이터 정규화
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 312 입니다.
x_test shape: (6777, 28, 28, 3)
y_test shape: (6777,)


In [18]:
score = model.evaluate(x_test_norm, y_test, verbose=1)
print('정답률 = ', score[1],'loss=', score[0])
#train데이터(류한웅, )

212/212 [==============================] - 1s 3ms/step - loss: 0.5198 - accuracy: 0.9731
정답률 =  0.9731444716453552 loss= 0.5197547674179077


# 회고
## input data의 channel이 왜 3이였는가? => 컬러사진이기 때문(RGB)
## 초기의 accuracy는 0.3%도 되지않았다. => 과적합
## 과적합이 된 원인은 내 사진이 비율로 따졌을때 거의 80프로를 차지하여서 일부 삭제를 해 다른 조원들의 사진과 비슷한 개수로 맞춰주어 해결하였다.
## ephocs를 너무 크게했을때도 accuracy가 낮게 나왔고, 학습률이 99프로에 가까워질때의 ephocs로 조정하니 문제가 사라졌다.
### train data는 나와 다른조원 2명의 이미지를 사용했고 test data는 또다른 조원 한명의 이미지를 사용했는데 의외로 높은 정답률을 나타내었다.